In [21]:
from abc import (
    ABC,
    abstractmethod,
    abstractclassmethod,
    abstractproperty
)
from dataclasses import dataclass
from typing import Generator, Optional
from IPython.display import display, HTML

import pygraphviz as pgv
import base64
import sympy as sm
from sympy.abc import a, b, c, d, e, f
from sympy.core.expr import Expr
from sympy.core.symbol import Symbol
from sympy.logic.boolalg import Boolean, BooleanTrue, BooleanFalse

In [22]:
sm.init_session()

IPython console for SymPy 1.14.0 (Python 3.12.4-64-bit) (ground types: python)

These commands were executed:
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.14.0/



# Abstract classes

In [23]:
@dataclass(frozen=True)
class Expression(ABC):

    @abstractproperty
    def name(self) -> str:
        ...

    @abstractclassmethod
    def is_final(cls) -> bool:
        ...

    @abstractmethod
    def to_sympy(self) -> Expr:
        ...


@dataclass(frozen=True)
class Operator(Expression, ABC):

    @abstractclassmethod
    def args_number(cls) -> int:
        ...

    @abstractproperty
    def args(self) -> tuple[Expression]:
        ...

    @classmethod
    def is_final(cls) -> bool:
        return False

    def __repr__(self):
        return f'{self.name}({", ".join(repr(arg) for arg in self.args)})'


@dataclass(frozen=True)
class Function(Operator, ABC):

    def __str__(self):
        return f'{self.name}({", ".join(
            str(arg) 
            if arg.is_final() or isinstance(arg, Function)
            else f"({arg})"
            for arg in self.args
        )})'

    def to_sympy(self) -> Expr:
        return sm.Function(self.name)(**list(arg.to_sympy()
                                             for arg in self.args))


@dataclass(frozen=True)
class InfixOperator(Operator, ABC):

    def __str__(self):
        return f" {self.name} ".join(
            str(arg)
            if arg.is_final() or isinstance(arg, Function)
            else f"({arg})"
            for arg in self.args
        )


@dataclass(frozen=True)
class Literal(Expression):
    _name: str

    @property
    def name(self) -> str:
        return self._name

    @classmethod
    def is_final(cls) -> bool:
        return True

    def __str__(self):
        return self.name

    def __repr__(self):
        return f'Literal({repr(self.name)})'

    def to_sympy(self) -> Expr:
        return Symbol(self.name)


class Rule(ABC):

    def __new__(cls):
        raise Exception("Static class. Initialization is not allowed.")

    @abstractclassmethod
    def can_apply(cls, expression: Expression) -> bool:
        ...

    @abstractclassmethod
    def apply(cls, expression: Expression) -> Expression:
        ...

# More classes

In [24]:
@dataclass(frozen=True)
class Addition(InfixOperator):
    _arg1: Expression
    _arg2: Expression

    @classmethod
    def args_number(cls) -> int:
        return 2

    @property
    def args(self) -> tuple[Expression]:
        return self._arg1, self._arg2

    @property
    def name(self) -> str:
        return '+'

    def to_sympy(self) -> Expr:
        return sm.Add(self._arg1.to_sympy(), self._arg2.to_sympy())


@dataclass(frozen=True)
class Multiplication(InfixOperator):
    _arg1: Expression
    _arg2: Expression

    @classmethod
    def args_number(cls) -> int:
        return 2

    @property
    def args(self) -> tuple[Expression]:
        return self._arg1, self._arg2

    @property
    def name(self) -> str:
        return '*'

    def to_sympy(self) -> Expr:
        return sm.Mul(self._arg1.to_sympy(), self._arg2.to_sympy())


@dataclass(frozen=True)
class Increment(Function):
    _arg: Expression

    @classmethod
    def args_number(cls) -> int:
        return 1

    @property
    def args(self) -> tuple[Expression]:
        return self._arg,

    @property
    def name(self) -> str:
        return 's'


@dataclass(frozen=True)
class Double(Function):
    _arg: Expression

    @classmethod
    def args_number(cls) -> int:
        return 1

    @property
    def args(self) -> tuple[Expression]:
        return self._arg,

    @property
    def name(self) -> str:
        return 'd'


_0 = Literal('0')


class RuleA(Rule):

    @classmethod
    def can_apply(cls, expression: Expression) -> bool:
        return isinstance(expression, Addition) \
            and expression.args[1] == _0

    @classmethod
    def apply(cls, expression: Expression) -> Expression:
        if cls.can_apply(expression):
            return expression.args[0]


class RuleB(Rule):

    @classmethod
    def can_apply(cls, expression: Expression) -> bool:
        return isinstance(expression, Addition) \
            and isinstance(expression.args[1], Increment)

    @classmethod
    def apply(cls, expression: Expression) -> Expression:
        if cls.can_apply(expression):
            x = expression.args[0]
            y = expression.args[1].args[0]
            return Increment(Addition(x, y))


class RuleC(Rule):

    @classmethod
    def can_apply(cls, expression: Expression) -> bool:
        return isinstance(expression, Multiplication) \
            and expression.args[1] == _0

    @classmethod
    def apply(cls, expression: Expression) -> Expression:
        if cls.can_apply(expression):
            return _0


class RuleD(Rule):

    @classmethod
    def can_apply(cls, expression: Expression) -> bool:
        return isinstance(expression, Multiplication) \
            and isinstance(expression.args[1], Increment)

    @classmethod
    def apply(cls, expression: Expression) -> Expression:
        if cls.can_apply(expression):
            x = expression.args[0]
            y = expression.args[1].args[0]
            return Addition(
                Multiplication(x, y), x
            )


class RuleE(Rule):

    @classmethod
    def can_apply(cls, expression: Expression) -> bool:
        return isinstance(expression, Double) \
            and expression.args[0] == _0

    @classmethod
    def apply(cls, expression: Expression) -> Expression:
        if cls.can_apply(expression):
            return _0


class RuleF(Rule):

    @classmethod
    def can_apply(cls, expression: Expression) -> bool:
        return isinstance(expression, Double) \
            and isinstance(expression.args[0], Increment)

    @classmethod
    def apply(cls, expression: Expression) -> Expression:
        if cls.can_apply(expression):
            x = expression.args[0].args[0]
            return Increment(Increment(Double(x)))

In [28]:
f1 = Multiplication(Literal('x'), Increment(_0))
print(f1)
f11 = RuleD.apply(f1)
print(f11)
print(RuleA.apply(f11))

x * s(0)
(x * 0) + x
None
